In [ ]:
import tensorflow as tf  # Importa la biblioteca TensorFlow

# Imprime la versión de TensorFlow instalada
print("Tensorflow version " + tf.__version__)

# Lista los dispositivos físicos disponibles (GPU, CPU, etc.)
print("Devices ", tf.config.list_physical_devices())

try:
    # Intenta detectar si hay una TPU disponible
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on a TPU with {tpu.num_accelerators()["TPU"]} cores')
except ValueError:
    # Si no se encuentra una TPU, lanza un error indicando que no está conectado a una TPU
    raise BaseException('ERROR: Not connected to a TPU runtime; please check the previous cell for instructions!')

# Conecta la configuración de TensorFlow al cluster de la TPU detectada
tf.config.experimental_connect_to_cluster(tpu)

# Inicializa el sistema TPU
tf.tpu.experimental.initialize_tpu_system(tpu)

# Define la estrategia de distribución para utilizar la TPU en la ejecución de modelos
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
# Instalación de dependencias necesarias
!pip install vit-keras tensorflow-addons optuna

In [ ]:
# Manejo de archivos, datos y memoria
import os
import random
import math
import gc
from datetime import datetime

# Procesamiento y visualización de datos
import numpy as np
import matplotlib.pyplot as plt

# Machine Learning y Preprocesamiento
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Vision Transformer (ViT) SOLO DISPONIBLE HASTA LA VERSION 2.15 DE TENSORFLOW
from vit_keras import vit

# Optuna para optimización de hiperparámetros
import optuna

# Google Colab (solo si se ejecuta en Colab)
from google.colab import drive, runtime

# Montar Google Drive (solo si se ejecuta en Colab)
drive.mount('/content/drive')

In [4]:
# Definir parámetros globales
IMG_SIZE = 224  # Tamaño de las imágenes de entrada (224x224 píxeles)
BATCH_SIZE = 64  # Tamaño del batch, recomendado entre 16 y 128
EPOCHS = 100  # Número máximo de épocas; Early Stopping finalizará antes si es necesario

# Configuración de la base de datos
TRAIN_ZIP_URL = "url/de/la/base/de/datos/"  # URL donde se encuentra la base de datos comprimida en .zip
DATASET_DIR = "/content/dataset"  # Directorio donde se extraerá la base de datos


In [ ]:
# Descargar el archivo .zip que contiene la base de datos (en este caso desde Dropbox)
!wget -O /content/dataset.zip $TRAIN_ZIP_URL

# Extraer el contenido del archivo .zip en el directorio especificado
!unzip /content/dataset.zip -d $DATASET_DIR

In [ ]:
# Directorios de las imágenes
train_dir = '/content/dataset/train'  # Directorio de imágenes de entrenamiento
val_dir = '/content/dataset/val'  # Directorio de imágenes de validación

# Crear datasets de entrenamiento y validación
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',  # Detecta automáticamente las subcarpetas como etiquetas (ej: benign, malignant)
    label_mode='int',  # Etiquetas en formato entero (0 para benign, 1 para malignant)
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),  # Ajusta el tamaño de imagen a 224x224
    shuffle=True  # Mezcla las imágenes aleatoriamente
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True
)

# Normalización de imágenes (Escala los píxeles de 0 a 1)
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Convierte valores de píxeles de [0,255] a [0,1]
    return image, label

# Aplicar normalización a los datasets
train_dataset = train_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

# Optimización del rendimiento
AUTOTUNE = tf.data.AUTOTUNE

# Cachear, barajar y pre-cargar datos para mejorar la eficiencia
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
# Definir el modelo basado en Vision Transformer (ViT)
def create_vit_model(img_size, dense_units, dropout_rate, l2_rate):
    vit_base = vit.vit_b16(
        image_size=img_size,
        pretrained=True,  # Usa pesos preentrenados
        include_top=False,  # Excluye la capa final preentrenada
        pretrained_top=False,
    )
    inputs = Input(shape=(img_size, img_size, 3))

    x = vit_base(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(dense_units, activation="relu",
                     kernel_regularizer=tf.keras.regularizers.l2(l2_rate))(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    return Model(inputs, outputs)

# Definir Focal Loss con ajuste de alpha
def focal_loss(gamma=2.0, alpha=0.75):
    def loss(y_true, y_pred):
        bce = tf.keras.losses.BinaryCrossentropy(
            reduction=tf.keras.losses.Reduction.NONE
        )(y_true, y_pred)
        pt = tf.exp(-bce)
        focal_loss_value = alpha * (1.0 - pt)**gamma * bce
        return tf.reduce_mean(focal_loss_value)
    return loss

# Función objetivo para Optuna (optimización de hiperparámetros)
def objective(trial):
    # Definir los hiperparámetros a optimizar
    dense_units = trial.suggest_int('dense_units', 128, 512)
    dropout_rate = trial.suggest_float('dropout_rate', 0.4, 0.7)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)
    l2_rate = trial.suggest_float('l2_rate', 1e-6, 1e-2, log=True)

    print(f"\n>>> Trial {trial.number} -- "
          f"dense_units={dense_units}, "
          f"dropout_rate={dropout_rate:.4f}, "
          f"learning_rate={learning_rate:.6f}, "
          f"l2_rate={l2_rate:.6f}")

    # Crear y compilar el modelo dentro del ámbito de TPU
    with tpu_strategy.scope():
        model = create_vit_model(
            img_size=IMG_SIZE,
            dense_units=dense_units,
            dropout_rate=dropout_rate,
            l2_rate=l2_rate,
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=focal_loss(alpha=0.75),  # Focal Loss ajustada para mejorar el recall
            metrics=[
                BinaryAccuracy(name="accuracy"),
                Precision(name="precision"),
                Recall(name="recall"),
                AUC(name="auc"),
            ],
        )

    # Callbacks para optimizar el entrenamiento
    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    checkpoint = ModelCheckpoint(
        filepath=f"/content/drive/MyDrive/Melanoma/Models/Model_v4/Model_v4.3.3(VIT)/Optuna/{trial.number}_optuna_checkpoint_model_v4.3.keras",
        save_best_only=True,
        save_weights_only=False,
        monitor="val_accuracy",
        mode="max"
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-7
    )

    # Pesos de clase ajustados
    class_weights = {0: 0.7, 1: 1.3}

    # Entrenar el modelo
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=[early_stopping, checkpoint, reduce_lr],
        verbose=1
    )

    # Retornar la mejor val_accuracy como métrica objetivo
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy

# Se puede cambiar la métrica objetivo para priorizar el recall si es necesario.

In [ ]:
# Crear un estudio Optuna con almacenamiento en SQLite
study = optuna.create_study(
    study_name="model_v4.3_ViT_study",  # Nombre del estudio
    storage="sqlite:////content/drive/MyDrive/Melanoma/Models/Model_v4/Model_v4.3.3(VIT)/Optuna/model_v4.3_ViT_study.db",  # Ruta donde se guardará el progreso
    direction='maximize',  # Optimiza para maximizar la métrica objetivo (val_accuracy)
    load_if_exists=True  # Si el estudio ya existe, lo carga en lugar de crearlo desde cero
)

# Ejecutar la optimización con 25 intentos (trials)
study.optimize(objective, n_trials=25)

# Mostrar los mejores hiperparámetros encontrados
print("Best hyperparameters:", study.best_params)
print(f"Best trial: {study.best_trial.number}")
print(f"Best value (val_accuracy): {study.best_trial.value}")

In [ ]:
# Liberar memoria y desconectar la sesión en Google Colab
runtime.unassign()